In [1]:
import pandas as pd
pd.set_option('display.max_colwidth', -1) 
import seaborn as sns
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import WhitespaceTokenizer
from nltk.corpus import wordnet
import re
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.naive_bayes import GaussianNB
import pickle
from collections import Counter
from itertools import chain
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import plot_confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  


In [ ]:
df = pd.read_csv('/content/new_file.csv')

In [ ]:
df.head(5)

,Unnamed: 0,target,timestamp,username,text,text_lemmatized,text_nostopwords,text_stemmed,low_removed
0,0,0,Mon Apr 06 22:19:45 PDT 2009,_TheSpecialOne_,switchfoot httptwitpiccom2y1zl awww thats a bummer you shoulda got david carr of third day to do it d,"['switchfoot', 'httptwitpiccom2y1zl', 'awww', 'thats', 'a', 'bummer', 'you', 'shoulda', 'get', 'david', 'carr', 'of', 'third', 'day', 'to', 'do', 'it', 'd']",switchfoot httptwitpiccom2y1zl awww thats bummer get david carr third day,switchfoot httptwitpiccom2y1zl awww thats bummer get david carr third day,switchfoot awww thats bummer get david carr third day
1,1,0,Mon Apr 06 22:19:49 PDT 2009,scotthamilton,is upset that he cant update his facebook by texting it and might cry as a result school today also blah,"['be', 'upset', 'that', 'he', 'cant', 'update', 'his', 'facebook', 'by', 'texting', 'it', 'and', 'might', 'cry', 'a', 'a', 'result', 'school', 'today', 'also', 'blah']",upset cant update facebook texting might cry result school today also blah,upset cant update facebook texting might cry result school today also blah,upset cant update facebook texting might cry result school today also blah
2,2,0,Mon Apr 06 22:19:53 PDT 2009,mattycus,kenichan i dived many times for the ball managed to save 50 the rest go out of bounds,"['kenichan', 'i', 'dive', 'many', 'time', 'for', 'the', 'ball', 'manage', 'to', 'save', '50', 'the', 'rest', 'go', 'out', 'of', 'bound']",kenichan dive many time ball manage save 50 rest go bound,kenichan dive many time ball manage save 50 rest go bound,dive many time ball manage save 50 rest go bound
3,3,0,Mon Apr 06 22:19:57 PDT 2009,ElleCTF,my whole body feels itchy and like its on fire,"['my', 'whole', 'body', 'feel', 'itchy', 'and', 'like', 'it', 'on', 'fire']",whole body feel itchy like fire,whole body feel itchy like fir,whole body feel itchy like fir
4,6,0,Mon Apr 06 22:20:03 PDT 2009,mybirch,need a hug,"['need', 'a', 'hug']",need hug,need hug,need hug


In [ ]:
df.drop(columns= 'Unnamed: 0', axis=1, inplace=True)

In [ ]:
df.columns

Index(['target', 'timestamp', 'username', 'text', 'text_lemmatized',
       'text_nostopwords', 'text_stemmed', 'low_removed'],
      dtype='object')

In [ ]:
X = df['low_removed']
y = df['target']

In [ ]:
X

0         switchfoot awww thats bummer get david carr third day                     
1         upset cant update facebook texting might cry result school today also blah
2         dive many time ball manage save 50 rest go bound                          
3         whole body feel itchy like fir                                            
4         need hug                                                                  
            ...                                                                     
599995    garden                                                                    
599996    heard first girl hope look wendys brain kid bab                           
599997    glad ur wel                                                               
599998    ready mojo makeover ask detail                                            
599999    happy                                                                     
Name: low_removed, Length: 600000, dtype: object

In [ ]:
vectorizer = TfidfVectorizer(max_df = 40, min_df = 5)

In [ ]:
X_vect = vectorizer.fit_transform(X.values.astype(str))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_vect,y,test_size=0.33, random_state=42) 

In [ ]:
X_train.shape

(402000, 10743)

In [ ]:
svd = TruncatedSVD(n_components=1000, random_state=42)
svd.fit(X_train)

TruncatedSVD(n_components=1000, random_state=42)

In [ ]:
X_train_svd = svd.transform(X_train)

In [ ]:
X_test_svd = svd.transform(X_test)

In [ ]:
#print(svd.explained_variance_ratio_)

In [ ]:
sum(svd.explained_variance_ratio_)

0.17669578410976364

In [ ]:
type(X_train_svd)

numpy.ndarray

In [ ]:
X_train_svd.shape

(402000, 1000)

In [ ]:
X_train_svd = X_train_svd.tolist()

In [ ]:
type(X_train_svd)

list

In [ ]:
X_train_svd = np.asarray(X_train_svd)

## Grid Search on Decision Tree


In [ ]:
dec_tree = DecisionTreeClassifier(random_state = 42)
dec_tree.fit(X_train_svd, y_train)
dec_tree.score(X_test_svd, y_test)

0.5286161616161617

In [ ]:
grid = {
    'max_depth': [2, 3, 5, 10, 20],
    'min_samples_leaf': [5, 10, 20, 50, 100]
}

In [ ]:
dec_tree_cv = GridSearchCV(estimator = dec_tree, 
                            param_grid = grid, 
                            cv = 4, verbose = 1, scoring = "f1")

In [ ]:
dec_tree_cv.fit(X_train_svd,y_train)

Fitting 4 folds for each of 25 candidates, totalling 100 fits


GridSearchCV(cv=4, estimator=DecisionTreeClassifier(random_state=42),
             param_grid={'max_depth': [2, 3, 5, 10, 20],
                         'min_samples_leaf': [5, 10, 20, 50, 100]},
             scoring='f1', verbose=1)

In [ ]:
print("tuned hyperparameters :(best parameters) ",dec_tree_cv.best_params_)
print("accuracy :",dec_tree_cv.best_score_)

tuned hyperparameters :(best parameters)  {'max_depth': 2, 'min_samples_leaf': 5}
accuracy : 0.3037582237378247
